
# The Apache Spark Scala API

## 1. Introduction

This notebook shows how to connect Jupyter notebooks to a Spark cluster to process data using Spark Scala API.

## 2. The Spark Cluster

### 2.1. Get Spark

Let's start by importing Apache Spark from Maven repository (mind the Spark **version**).

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.0.0`;

Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sql_2.12/3.0.0/spark-sql_2.12-3.0.0.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/almond-spark_2.12/0.10.9/almond-spark_2.12-0.10.9.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-parent_2.12/3.0.0/spark-parent_2.12-3.0.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/apache/18/apache-18.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-databind/2.10.0/jackson-databind-2.10.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sketch_2.12/3.0.0/spark-sketch_2.12-3.0.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-catalyst_2.12/3.0.0/spark-catalyst_2.12-3.0.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-core_2.12/3.0.0/spark-core_2.12-3.0.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/xbean/xbean-asm7-shaded/4.15/xbean-asm7-shaded-4.15.pom
Downloaded https://repo1.maven.org/mav

import $ivy.$                                  ;

We will be disabling Spark internal logs to let us focus on its API.

In [2]:
import org.apache.log4j.{Level, Logger};
Logger.getLogger("org").setLevel(Level.OFF);

import org.apache.log4j.{Level, Logger};


### 2.2. Connection

To connect to the Spark cluster, create a SparkSession object with the following params:

+ **appName:** application name displayed at the [Spark Master Web UI](http://localhost:8080/);
+ **master:** Spark Master URL, same used by Spark Workers;
+ **spark.executor.memory:** must be less than or equals to docker compose SPARK_WORKER_MEMORY config.

In [3]:
import org.apache.spark.sql._

val spark = SparkSession.
            builder().
            appName("scala-spark-notebook").
            master("spark://spark-master:7077").
            config("spark.executor.memory", "512m").
            getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@721a5278

More confs for SparkSession object in standalone mode can be added using the **config** method. Checkout the API docs [here](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/SparkSession.html).

## 3. The Data

### 3.1. Introduction

We will be using Spark Scala API to read, process and write data. Checkout the API docs [here](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/index.html).

### 3.2. Read

Let's read some UK's macroeconomic data ([source](https://www.kaggle.com/bank-of-england/a-millennium-of-macroeconomic-data)) from the cluster's simulated **Hadoop distributed file system (HDFS)** into a Spark dataframe.

In [4]:
val data = spark.read.format("csv").option("sep", ",").option("header", "true").load("data/uk-macroeconomic-data.csv")

: 

Let's then display some dataframe metadata, such as the number of rows and cols and its schema (cols name and type).

In [ ]:
data.count

In [ ]:
data.columns.size

In [ ]:
data.printSchema

### 3.3. Process

In this example, we will get UK's population and unemployment rate thoughtout the years. Let's start by selecting the relevant columns.

In [ ]:
var unemployment = data.select("Description", "Population (GB+NI)", "Unemployment rate")

In [ ]:
unemployment.show(10)

We successfully selected the desired columns but two problems were found:
+ The first line contains no data but the unit of measurement of each column;
+ There are many years with missing population and unemployment data.

Let's then remove the first line.

In [ ]:
val cols_description = unemployment.filter(unemployment("Description") === "Units")

In [ ]:
cols_description.show()

In [ ]:
unemployment = unemployment.join(cols_description, unemployment("Description") === cols_description("Description"), "left_anti")

In [ ]:
unemployment.show(10)

Nice! Now, let's drop the dataframe rows with missing data and refactor its columns names.

In [ ]:
unemployment = unemployment.na.drop()

In [ ]:
unemployment = unemployment.
                withColumnRenamed("Description", "year").
                withColumnRenamed("Population (GB+NI)", "population").
                withColumnRenamed("Unemployment rate", "unemployment_rate")

In [ ]:
unemployment.show(10)

### 3.4. Write

Lastly, we persist the unemployment data into the cluster's simulated **HDFS**.

In [ ]:
unemployment.repartition(1).write.format("csv").mode("overwrite").option("sep", ",").option("header", "true").save("data/uk-macroeconomic-unemployment-data.csv")